In [2]:
using ConstLab
using Plots
using Voigt
plotly();

In [10]:
include("mises_viscoplast.jl");

### Parameters

In [11]:
E = 200.0 * 10^9
ν =  0.3
σy = E / 1e3
H = 0.1 * E
n = 1.5
σc =  0.1 * 200*10^9
t_star = 1.0
mp = ViscoPlastMP(E, ν, σy, H, n, σc, t_star)

ViscoPlastMP
  E: 2.0e11
  ν: 0.3
  σy: 2.0e8
  H: 2.0e10
  n: 1.5
  σc: 2.0e10
  t★: 1.0

### Material Status

In [12]:
ms = ViscoPlastMS()

ViscoPlastMS
  ₙεₚ: [0.0,0.0,0.0,0.0,0.0,0.0]
  ₙσ: [0.0,0.0,0.0,0.0,0.0,0.0]
  ₙκ: 0.0
  ₙμ: 0.0

### Loadcase: Uniaxial Strain

In [18]:
α = 25.0
tend = α * t_star
nts = 100
ts = collect(linspace(0, tend, nts))
ε_max = 0.005
εs_pstrain, σs_pstrain, ε_control_pstrain = loadcase(:uniaxial_strain, ε_max, ts);

### Driver

In [15]:
εs_res_pstrain, σs_res_pstrain, matstats = driver(stress, ATS, ms, mp, ts, εs_pstrain, 
σs_pstrain, ε_control_pstrain);

### Loadcase: Uniaxial Stress

In [16]:
εs_pstress, σs_pstress, ε_control_pstress = loadcase(:uniaxial_stress, ε_max, ts);
εs_res_pstress, σs_res_pstress, matstats = driver(stress, ATS, ms, mp, ts, εs_pstress, 
                                                  σs_pstress, ε_control_pstress, ftol = 1e-5);

### Plotting

In [17]:
vm(x) = vnorm(dev(x))

vm (generic function with 1 method)

In [31]:
p = plot(vec(εs_res_pstrain[1,:]), 
Float64[vm(σs_res_pstrain[:,i]) for i = 1:nts],
label="Plane strain");
plot!(p, vec(εs_res_pstress[1,:]), 
         Float64[vm(σs_res_pstress[:,i]) for i = 1:nts],
label="Plane stress");

In [32]:
p